# Simplex

#### Katherine Yohanna Mazariegos Guerra
#### Abner Xocop Chacach

In [1]:
import numpy as np

In [2]:
# En archivo exercise01.txt x1, x2 y x3 es el nombre de las variables, se usaron esas para seguir con el estándar de nombramiento en Programación Lineal.
# Es necesario escribir si es min/max para que el programa identifique si es de maximización o minimización, st para indicar que esas son las restricciones y end para indicar al programa que el problema termina ahí

def parse_coefficients(coefficient_list, monomial):
    """
    Este es un parseador de coeficientes. Consiste en comprobar si una cadena tiene una expresión regular en donde pueda extraer caracteres específicos, en este caso se busca extraer los coeficientes.

    Args:
        :rtype: None
        :param coefficient_list: Lista en la que se almacenarán los coeficientes
        :param monomial: Una cadena (por ejemplo, -3x1) que será analizada hasta su coeficiente (por ejemplo, -3)

    Verifica qué patrón coincide. Válidos son: (s)(n)lv
        Los paréntesis indican la existencia opcional
        s es + o - (la ausencia significa +)
        n es un número (coeficiente, la ausencia significa 1)
        l es una letra latina minúscula (letra variable)
        v es un número, probablemente incremental (número variable)

    Import re:
        Una expresión regular (o RE) especifica un conjunto de cadenas que se corresponde con ella; las funciones de este módulo le permiten comprobar si una cadena particular se corresponde con una expresión regular dada (o si una expresión regular dada se corresponde con una cadena particular, que se reduce a lo mismo)
        Source: https://docs.python.org/3/library/re.html
    """
    import re

    if re.match('[ ]*[\+ ]?[\d]+[\.]?[\d]*', monomial):
        float_cast = float(re.match('[ ]*[\+ ]?[\d]+[\.]?[\d]*', monomial).group(0))
        coefficient_list.append(float_cast)
    elif re.match('[ ]*[\-][\d]+[\.]?[\d]*', monomial):
        float_cast = float(re.match('[ ]*[\-][\d]+[\.]?[\d]*', monomial).group(0))
        coefficient_list.append(float_cast)
    elif re.match('[ ]*[\+]*[a-z][\d]+', monomial):
        coefficient_list.append(1)
    elif re.match('[ ]*[\-][a-z][\d]+', monomial):
        coefficient_list.append(-1)

In [3]:
lines = []
def parse_lp1(input_filename):
    """
    Esta función es la encargada de leer el archivo, hará uso del parser para mandar línea a línea el contenido del .txt y según los coeficientes que obtenga devolverá las matrices y arrays correspondientes. 

    Tiene tareas como verificar que el archivo haya sido encontrado, leer si es un problema de maximización o minimización, llenar las matrices/ arrays.

    :rtype : tuple
    :param input_filename: Nombre de archivo de la entrada del problema lineal
    :return: Retorna A-matrix, b-vector, c-vector, MinMax
    """
    import re

    error = 0 # Inicializar la variable de error. Si el error!=0 entonces hubo un problema de entrada de archivo 


    try:
        infile = open('Nathalia06.txt')
    except FileNotFoundError:
        error = 1
        print('\nInput file error: Archivo no encontrado.') # Archivo no encontrado

    #lines = []
    if error != 1:
        for line in infile:
            lines.append(line)
        infile.close()
    for line in lines:
        print(line, end='')    

    minmax_line = '' # Verficar si el problema es de maximización o de minimización
    for line in lines:
        if re.match('^[ ]*max|min', line):
            minmax_line = line

    minmax = 0
    objective_function = ''

    if re.match('^[ ]*max', minmax_line): #Si en el archivo se encuentra la palabra 'max' entonces el problema es de maximización
        minmax = -1
        objective_function = minmax_line
        objective_function = objective_function.strip('max')
    
    elif re.match('^[ ]*min', minmax_line): # Si en el archivo se encuentra la palabra 'min' entonces el problema es de minimización
        minmax = 1
        objective_function = minmax_line
        objective_function = objective_function.strip('min')

    if minmax_line == '' and minmax == 0: # Si en el archivo no se encuentra ni 'max' ni  'min' entonces no hay función objetivo
        error = 2
        print('\nInput file error: Función objetivo no encontrada.')

    c_vector = [] # Rellenar el vector c con coeficientes de función objetiva

    regex = re.compile('^[\+\- ]?[\d]*[\.]?[\d]*[a-z][\d+]')
    while regex.match(objective_function):
        monomial = regex.match(objective_function).group(0)
        parse_coefficients(c_vector, monomial)
        objective_function = objective_function.replace(monomial, '', 1)

    a_matrix = [] # Rellenar la matriz A (coeficientes) y el vector b utilizando las restricciones del problema
    b_vector = []
    eqin = []

    st_line = ''
    st_index = 0
    for index, line in enumerate(lines):
        if 'st' in line:
            st_index = index
            st_line = line

    if re.match('^[ ]*st', st_line):
        st_line = st_line.replace('st', '  ', 1)

    if st_line == '':
        error = 3
        print('\nInput file error: Línea de restricciones no encontrada. No existe la keyword \'st\'.')

    while st_index < len(lines) - 1:
        sub_a_vector = []
        a_matrix.append(sub_a_vector)
        while True:
            st_line = st_line.strip(' ')
            if re.match('^[\+\- ]?[\d]*[\.]?[\d]*[a-z][\d+]', st_line):
                monomial = re.match('^[\+\- ]?[\d]*[\.]?[\d]*[a-z][\d+]', st_line).group(0)
                parse_coefficients(sub_a_vector, monomial)
                st_line = st_line.replace(monomial, '', 1)
            elif re.match('^[<>=]+', st_line):
                monomial = re.match('^[<>=]+', st_line).group(0)
                if monomial == '<=':
                    eqin.append(-1)
                elif monomial == '>=':
                    eqin.append(1)
                elif monomial == '==':
                    eqin.append(0)
                else:
                    error = 4
                    print('\nInput file error: Caracter inesperado; esperados <=, >=, = al menos', monomial)
                st_line = st_line.replace(monomial, '', 1)
            elif re.match('^[\d]+', st_line):
                monomial = re.match('^[\d]+', st_line).group(0)
                int_cast = int(re.match('^[\d]+', st_line).group(0))
                b_vector.append(int_cast)
                st_line = st_line.replace(monomial, '', 1)
            else:
                if not sub_a_vector:    # Evalúa true cuando las líneas están vacías entre las restricciones
                    a_matrix.pop()
                break

        st_index += 1 #         Incrementar el número de línea y obtener el siguiente
        st_line = lines[st_index]

        if st_line == 'end\n' and error == 0: #         Búsqueda de la declaración final y ausencia de errores
            print('\nArchivo cargado exitosamente.')
            break

    return a_matrix, b_vector, c_vector, eqin, minmax # Devolver todas las listas y variables creadas

In [4]:
def convert_to_dual(input_filename, output_filename):
    """
    Verifica si son restricciones de >=, <= o =. También tiene como tarea hacer un archivo de salida en el que muestre los resultados de las matrices que se llenaron.

    :param input_filename: Nombre de archivo de la entrada del problema lineal
    :param output_filename: Filename of the linear problem output
    :return: Returns A-matrix, b-vector, c-vector, Variable-constraints, MinMax
    """

    (a_matrix, b_vector, c_vector, eqin, minmax) = parse_lp1(input_filename)     # Llamar la función parse_lp1

    variable_constraints = [] # Convertir las restricciones a equivalentes duales '*' significa libre
    if minmax == -1:
        for el in eqin:
            if el == 0:
                variable_constraints.append('==')
            elif el == 1:
                variable_constraints.append('>=')
            elif el == -1:
                variable_constraints.append('<=')

    a_matrix = list(zip(a_matrix)) # Traspuesta de A-matrix

    minmax = -minmax # min(max) el problema dual es max(min)

    outfile = open(output_filename, 'w') # Escribir el problema a un archivo de salida
    outfile.write('(Objective Function) b-vector: [' + ', '.join(map(str, b_vector)) + ']\n')

    outfile.write('\nA-matrix: [')
    thing = ''
    for index, sub_a_vector in enumerate(a_matrix):
        thing += '[ ' + ', '.join(map(str, sub_a_vector)) + ']'
        if index != (len(a_matrix) - 1):
            thing += ', '
    outfile.write(thing + ']\n')

    outfile.write('\n(Contraints) c-vector: [' + ', '.join(map(str, c_vector)) + ']\n')
    outfile.write('\n(Variable Contraints) variable_constraints-vector: [' + ', '.join(map(str, c_vector)) + ']\n')
    outfile.write('\nEqin: [' + ', '.join(map(str, eqin)) + ']\n')
    outfile.write('\nMinMax: [' + str(minmax) + ']\n')
    outfile.close()

    return a_matrix, b_vector, c_vector, variable_constraints, eqin, minmax

(a_matrix, b_vector, c_vector, variable_contraints, eqin, minmax) = convert_to_dual('input-lp1', 'output-lp2')


max -2x1-3x2
st 5x1+4x2<=32
   1x1+2x2<=10
end

In [5]:
print(a_matrix)
print(b_vector)
print(c_vector)
print(variable_contraints)

[([5.0, 4.0],), ([1.0, 2.0],)]
[32, 10]
[]
['<=', '<=']


In [6]:
#Hacer matriz
#aqui estan los coeficientes de las restricciones
"""
Aqui se hace la matriz con las slack variables y con las variables artificiales que sean necesarias
"""
positions = []
a_matrix2 = []
for i in a_matrix:
    a_matrix2.append((i[0]))
#print(a_matrix2[0])
#convertimos a_matrix2 en una matriz de numpy para hacer operaciones con ella
mat = np.asmatrix(a_matrix2)
size = len(variable_contraints)
for i in range (len(variable_contraints)):
    if variable_contraints[i] == "<=":
        #hacemos la columna de ceros, de tamaño igual 
        # a las columnas que están en la matrix (length del arreglo de símbolos)
        new_matrix = np.asmatrix(np.zeros(size))
        #colocamos 1 en el espacio correspondiente de la fila en donde está 
        # la constrait
        new_matrix[0,i]=1
        #asignamos la forma para que sea un vector columna
        new_matrix.shape = (size,1)
        # obtenemos las dimensiones de la matrix actual
        x, y = mat.shape
        # hacemos el append de la columna de ceros y el uno en la posición correspondiente
        # al final de la matriz, a la derecha
        mat = np.hstack((mat[:,:y], new_matrix, mat[:,y:]))        
        print(a_matrix2[i])
        print("menor")
    if variable_contraints[i] == "==":
        new_matrix = np.asmatrix(np.zeros(size))
        new_matrix[0,i]=1
        new_matrix.shape = (size,1)
        x, y = mat.shape
        mat = np.hstack((mat[:,:y], new_matrix, mat[:,y:]))  
        print(a_matrix2[i])
        print("igual")
        positions.append(y)
    if variable_contraints[i] == ">=":
        new_matrix = np.asmatrix(np.zeros(size))
        new_matrix[0,i]=1
        new_matrix1 = np.asmatrix(np.zeros(size))
        new_matrix1[0,i]=-1
        new_matrix.shape = (size,1)
        new_matrix1.shape = (size,1)
        x, y = mat.shape
        mat = np.hstack((mat[:,:y], new_matrix, mat[:,y:]))
        mat = np.hstack((mat[:,:y+1], new_matrix1, mat[:,y+1:]))
        print(a_matrix2[i])
        print("mayor")
        positions.append(y)
    #print(variable_contraints[i])
#print(variable_contraints[0])
print(mat)

#Numero de columnas en la matriz
num_cols = mat.shape[1]
print(num_cols)
print(positions)

[5.0, 4.0]
menor
[1.0, 2.0]
menor
[[5. 4. 1. 0.]
 [1. 2. 0. 1.]]
4
[]


In [7]:
"""
Solicita el número de restricciones para que el programa sepa las iteraciones que tiene que hacer para 
la matriz de restricciones
"""
no_restricciones = int(input("Ingrese el no. de restricciones: "))

Ingrese el no. de restricciones: 2


In [8]:
"""
Solicita el número de variables para que sepa la cantidad de columnas que tiene que tener el programa para crear una matriz
de restricciones.

Variables sirven para saber el número de columnas de la matriz
Restricciones sirven para saber el número de filas de la matriz
"""

no_variables = int(input("Ingrese el no. de variables: "))

Ingrese el no. de variables: 2


In [9]:
"""
Esto se hace porque el algoritmo simplex necesita esta información para que al introducir la matriz identidad sepa si los 1's 
son positivos o negativos.

Si es min, los unos serán negativos.
Si es max, los unos serán positivos.
"""

max_min = True
tipo = input("¿El problema es de maximización 'max' o minimización 'min'?  ")
if tipo == "min":
    max_min = False
print(max_min)  

¿El problema es de maximización 'max' o minimización 'min'?  min
False


In [10]:
"""
Aquí es la orquetastación principal, porque aquí es donde se miden el número de filas y columnas que tendrá la tableau.
Así mismo se inicializa el algoritmo simplex de manera explícita. También se le indica si es un problema tipo Min o Max.Luego
empieza a buscar los puntos pivote, apoyándose de la eliminación gaussiana.

Más adelante han sido definidas dos funciones: una de maximización, donde se le envía la palabra 'MAX' y esta función
reconoce que tiene que resolver una maximización. La otra función es de minimización, que envía la palabra 'MIN'
lo que significa que se requiere de una minimización.

Estas palabras permiten que el algoritmo prepare la tableau según lo requiere la maximización y la minimización, porque 
ambos tienen una resolución distinta.
"""

def simplex(of,basis,tableau,opt):
    # Obtiene el número de filas y columnas que tiene la tableau
    n_rows = tableau.shape[0]
    n_cols = tableau.shape[1]
    if opt =='MIN':
        # Inicia el algoritmo simplex
        # Calcula zj-cj. Si cj - zj >= 0 para todas las columnas, entonces la solución actual es la solución óptima.
        check = of - np.sum(np.reshape(of[list(basis)],(n_rows,1)) * tableau[:,0:n_cols-1],axis=0)
    else:
        # Inicia el algoritmo simplex
        # Calcula cj-zj. Si zj - cj >= 0 para todas las columnas, entonces la solución actual es la solución óptima.
        check = np.sum(np.reshape(of[list(basis)],(n_rows,1)) *tableau[:,0:n_cols-1],axis=0) - of
    count = 0
    while ~np.all(check >=0):
        print(check)
        # Determina la columna pivote: La columna pivotante es la columna correspondiente al mínimo zj-cj.
        pivot_col = np.argmin(check)
        # Determinar los elementos positivos en la columna pivote. Si no hay elementos positivos en la columna pivote, 
        # entonces la solución óptima no tiene límites.
        positive_rows = np.where(tableau[:,pivot_col] > 0)[0]
        if positive_rows.size == 0:
            print('*******SOLUCIÓN SIN LÍMITES******')
            break
        # Determinar la hilera de pivote: prueba de min-ration
        divide=(tableau[positive_rows,n_cols-1]
            /tableau[positive_rows,pivot_col])
        pivot_row = positive_rows[np.where(divide 
            == divide.min())[0][-1]]
        # Actualizar las bases
        basis[pivot_row] = pivot_col
        # Realizar la eliminación gaussiana para hacer girar el elemento uno y los elementos por encima y por debajo del cero:
        tableau[pivot_row,:]=(tableau[pivot_row,:]
            /tableau[pivot_row,pivot_col])
        for row in range(n_rows):
            if row != pivot_row:
                tableau[row,:] = (tableau[row,:] 
                    - tableau[row,pivot_col]*tableau[pivot_row,:])
        if opt =='MIN':
            check = of - np.sum(np.reshape(of[list(basis)],(n_rows,1)) * tableau[:,0:n_cols-1],axis=0)
        else:
            check = np.sum(np.reshape(of[list(basis)],(n_rows,1)) *tableau[:,0:n_cols-1],axis=0) - of
        count += 1
        print('Paso %d' % count)
        print(tableau)
    return basis,tableau

def get_solution(of,basis,tableau):
    # Obtiene el número de columnas de la tableau
    n_cols = tableau.shape[1]
    # Obtener la solución óptima
    solution = np.zeros(of.size)
    solution[list(basis)] = tableau[:,n_cols-1]
    # Determinar el valor óptimo
    value = np.sum(of[list(basis)] * tableau[:,n_cols-1])
    return solution,value

In [11]:
"""
Esta función es muy importante, ya que permite ingresar variables artificiales si son necesarias. Los símbolos que requieren
que se añadan variables artificiales son los signos de >=. El símbolo == requiere una variable artificial en vez de una de 
holgura, el signo >= requiere de una variable artificial 1 y de una variable de holgura -1.
"""

n_b_vector = []
for i in b_vector:
    list1 = [i]
    n_b_vector.append(list1)

#Esta es la matriz que si sirve 
matrix_mat = np.concatenate((mat, n_b_vector),axis =1)
print(matrix_mat)

[[ 5.  4.  1.  0. 32.]
 [ 1.  2.  0.  1. 10.]]


In [12]:
print(variable_contraints)

def check_availability(element, collection: iter):
    return element in collection

verificar_menoresque = check_availability('<=', variable_contraints)
print(verificar_menoresque)

verificar_mayoresque = check_availability('>=', variable_contraints)
print(verificar_mayoresque)

verificar_igualesque = check_availability('==', variable_contraints)
print(verificar_igualesque)

['<=', '<=']
True
False
False


In [13]:
"""
Esta función es utilizado por la maximización, por eso aquí se añaden los coeficientes de la función objetivo, y se añaden
la misma cantidad de ceros que las variables de holgura, por ejemplo si se añadieron 3 variables de holgura, esta identifica
que tiene que añadir 3 ceros. Pero si se añaden variables de holgura y artifiales, la suma de estas será la cantidad de ceros
que se añadan.
"""

matrix_cero = []
size_ceros = num_cols - no_variables
print(size_ceros)

for i in range(size_ceros):
    matrix_cero.append(0)
print(matrix_cero)

objective_matrix = np.concatenate((c_vector, matrix_cero), axis=0)
print(objective_matrix )

2
[0, 0]
[0. 0.]


In [14]:
"""
Esta función la hemos colocado, ya que nuestro algoritmo requiere que se le indican las variables de holgura, las cuales 
son la base de la resolución del caso. Este array con bases es importante para el buen funcionamiento de los pivotes,
ya que si no se le dice explícitamente cuáles son las bases, sería difícil para nuestro algoritmo determinar cuál columna
sale y cuál fila entra.
"""

array_con_bases = []
numero_base = no_variables
for item in range(no_restricciones):  
    array_con_bases.append(item + numero_base)    
print(array_con_bases)

[2, 3]


In [15]:
"""
Esta es la función de maximización. Nos hemos fijado que la maximización requieres que los coeficientes de la función objetivo
no sean negativos, por eso se le manda explícitamente a la función simplex la palabra 'MAX' que identifica la maximización.
Eso permite que se diferencie la minimización con la maximización, y haga el procedimiento adecuado según lo requiera
cada caso.
"""

def max_function():
    # Definir la tableau:
    tableau = np.array(matrix_mat)
    print(tableau)
    # Define la función objetivo y las bases iniciales
    print(objective_matrix)
    
    of = np.array(objective_matrix)
    # bases iniciales
    #basis = np.array([4,5,6])
    basis = np.array(array_con_bases)
    # Correr el algorithm simplex
    basis,tableau = simplex(of,basis,tableau,'MAX')
    # Obtener la solución óptima
    optimal_solution,optimal_value = get_solution(of,basis,tableau)
    # Imprimir al tableau final.
    print('La base final es:')
    print(basis)
    print('solución')
    for i in range(len(optimal_solution)):
        print('X%d'%(i+1),'=',optimal_solution[i])
    print('Z=',optimal_value)
#max_function()

In [16]:
"""
Esta función nos sirve para hacer un array con la función objetivo. Adicional a eso se requieren las posiciones en las que 
se encuentran los signos >= o ==, ya que estos requieren que existan variables artificiales, las cuales nuestro algoritmo
evalúa como números muy grandes, por eso en lugar de tener ceros o una M como variable artifial, necesitamos tener números muy 
grandes, es por eso que se le envía un número 10000, que es suficientemente grande.

Esta función es utilizado únicamente para la resolución de los problemas de minimización.
"""

#Aqui es donde hay que hacer el replace
#El arreglo postitions esta en donde hago la matriz arriba
print(positions)
matrix_cero2 = []
size_ceros = no_variables
print(size_ceros)

for i in range(size_ceros):
    matrix_cero2.append(0)
print(matrix_cero2)

objective_matrixmin = np.concatenate((c_vector, matrix_cero2), axis=0)


for i in range(len(objective_matrixmin)):
    for i in range (len(positions)):
        objective_matrixmin[positions[i]]=10000
        
print(objective_matrixmin)
print(len(objective_matrixmin))


[]
2
[0, 0]
[0. 0.]
2


In [19]:
"""
Esta es la función de minimización. En la solución del tableau, se le indica explícitamente que se hará una MIN (minimización).
Esta es mandada a la función 'simplex' definida unos pasos anteriores.

La diferencia de la minimización es que esta busca que en la función objetivo los coeficientes sean < 0
"""

def min_function():
    # Definir la tableau:
    tableau = np.array(matrix_mat)
    # Definir la función objetivo y las bases iniciales.
    print(objective_matrix)
    #of = np.array([3, 9, -10000, -10000, 0, 0])
    of = np.array(objective_matrixmin)
    # bases iniciales
    basis = np.array(array_con_bases)
    # corre el algoritmo simple
    basis,tableau = simplex(of,basis,tableau,'MIN')
    # Obtner la solución óptima
    optimal_solution,optimal_value = get_solution(of,basis,tableau)
    # Tableau final
    print('Las bases finales son')
    print(basis)
    print('Solución')
    for i in range(len(optimal_solution)):
        print('X%d'%(i+1),'=',optimal_solution[i])
    print('Z=',optimal_value)
#min_function()

In [20]:
"""
Teniendo las funciones de maximización o minimización anteriores, este código verifica cuál de los dos tiene que hacer, 
basándose en lo que le indicamos al inicio . SI le dijimos max, hará la Maximización, si le dijimos que era min, hará la Mini-
mización.

Al mismo tiempo, imprime la la solución optimizada.
"""

if max_min:
    print("==== MAXIMIZACION ====")
    max_function()
else:
    print("==== MINIMIZACION ====")
    min_function()

==== MINIMIZACION ====
[0. 0.]


IndexError: index 2 is out of bounds for axis 0 with size 2